# Converters

Converters are used to transform prompts before sending them to the target.

This can be useful for a variety of reasons, such as encoding the prompt in a different format, or adding additional information to the prompt. For example, you might want to convert a prompt to base64 before sending it to the target, or add a prefix to the prompt to indicate that it is a question.

## Simple Converter Example

Converters can be used to perform these types of transformations. Here is a simple program that uses Rot13Converter converter, RandomCapitalLettersConverter, and AsciiArtConverter.

In [ ]:
from pyrit.prompt_converter import (
    AsciiArtConverter,
    BinaryConverter,
    FirstLetterConverter,
    RandomCapitalLettersConverter,
    ROT13Converter,
)
from pyrit.setup import IN_MEMORY, initialize_pyrit_async

await initialize_pyrit_async(memory_db_type=IN_MEMORY)  # type: ignore

prompt = "tell me how to cut down a tree"


print(await ROT13Converter().convert_tokens_async(prompt=prompt))  # type: ignore
print(await RandomCapitalLettersConverter(percentage=25.0).convert_tokens_async(prompt=prompt))  # type: ignore
print(await AsciiArtConverter().convert_tokens_async(prompt=prompt))  # type: ignore
print(await BinaryConverter().convert_tokens_async(prompt=prompt))  # type: ignore
print(await FirstLetterConverter().convert_tokens_async(prompt=prompt))  # type: ignore

text: gryy zr ubj gb phg qbja n gerr
text: Tell Me hOw to cut DOwn a treE
text:  ____  ____  __    __          _  _  ____        _  _   __   _  _        ____   __          ___  _  _  ____        ____   __   _  _  __ _         __         ____  ____  ____  ____ 
(_  _)(  __)(  )  (  )        ( \/ )(  __)      / )( \ /  \ / )( \      (_  _) /  \        / __)/ )( \(_  _)      (    \ /  \ / )( \(  ( \       / _\       (_  _)(  _ \(  __)(  __)
  )(   ) _) / (_/\/ (_/\      / \/ \ ) _)       ) __ ((  O )\ /\ /        )(  (  O )      ( (__ ) \/ (  )(         ) D ((  O )\ /\ //    /      /    \        )(   )   / ) _)  ) _) 
 (__) (____)\____/\____/      \_)(_/(____)      \_)(_/ \__/ (_/\_)       (__)  \__/        \___)\____/ (__)       (____/ \__/ (_/\_)\_)__)      \_/\_/       (__) (__\_)(____)(____)

text: 0000000001110100 0000000001100101 0000000001101100 0000000001101100 0000000000100000 0000000001101101 0000000001100101 0000000000100000 0000000001101000 0000000001101111 0000000001110111 00

In [ ]:
# Close connection
from pyrit.memory import CentralMemory

memory = CentralMemory.get_memory_instance()
memory.dispose_engine()